# CaptionAsset — Auto-Generated & Animated Subtitles
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/video-db/videodb-cookbook/blob/main/editor/feature/caption_asset_auto_subtitles.ipynb)

Welcome to the **CaptionAsset** feature notebook.

In this notebook, you'll learn how to add professional, animated subtitles to your videos programmatically using VideoDB Editor.

## What is CaptionAsset?

CaptionAsset is a specialized asset type designed for subtitles and closed captions. Unlike TextAsset (which is for static text overlays), CaptionAsset is built specifically for time-synced captions with powerful features:

- **Auto-Generation**: Set `src="auto"` to automatically transcribe speech and generate subtitles
- **Animations**: Dynamic text effects like karaoke highlighting, word-by-word reveals, and active word emphasis
- **ASS Styling**: Uses Advanced SubStation Alpha format for professional subtitle styling
- **Time-Synced**: Captions automatically align with spoken words in your video

## What You'll Learn

By the end of this notebook, you'll know how to:

1. Auto-generate subtitles from video speech
2. Style captions with custom fonts, colors, and positioning
3. Apply four different caption animations (reveal, karaoke, supersize, box_highlight)
4. Use ASS color format for caption styling
5. Create professional-looking animated subtitles for any video

Let's get started!

---

## 📦 Step 1: Installation

First, install the VideoDB SDK.

In [1]:
%pip -q install git+https://github.com/video-db/videodb-python.git@ankit/add-videodb-editor

  Preparing metadata (setup.py) ... done


---

## 📦 Step 2: Imports

Now let's import everything we need for working with captions:

- **Core Editor Classes**: Timeline, Track, Clip, VideoAsset
- **Caption Classes**: CaptionAsset, CaptionAnimation, CaptionAlignment
- **Styling Classes**: FontStyling, Positioning, BorderAndShadow, CaptionBorderStyle
- **Playback**: play_stream to preview our videos

In [2]:
import videodb

from videodb import play_stream
from videodb.editor import (
    Timeline, Track, Clip, VideoAsset,
    CaptionAsset, CaptionAnimation, CaptionAlignment, CaptionBorderStyle,
    FontStyling, Positioning, BorderAndShadow
)

---

## 📦 Step 3: Connect to VideoDB

Enter your VideoDB API key to establish a connection.

You don't need to modify this code—just run it and paste your API key when prompted.

In [3]:
import os
from getpass import getpass

api_key = getpass("Please enter your VideoDB API Key: ")
os.environ["VIDEO_DB_API_KEY"] = api_key

conn = videodb.connect()
coll = conn.get_collection()

print("✅ Connected to VideoDB")

Please enter your VideoDB API Key: ··········
✅ Connected to VideoDB


---

## 📦 Step 4: Upload Videos

For this notebook, we'll upload several videos to demonstrate different caption animations.

**Important**: Choose videos with clear speech for best auto-caption results. Narration, interviews, or dialogue work great.

Replace the YouTube URLs below with your own videos if desired:

In [4]:
# Upload the video for basic captions
video = coll.upload(url="https://www.youtube.com/watch?v=cqrJzG03ENE")
print(f"✅ Video uploaded: {video.id}")

✅ Video uploaded: m-z-019b4d9c-c538-79e2-b767-3fdc43b1b7b8


In [ ]:
# If you need to reconnect to an already uploaded video, use:
# video = coll.get_video("your_video_id_here")

### Lets review the original video without subtitles


In [6]:
# Initialize timeline with neutral gray background
timeline = Timeline(conn)
timeline.background = "#2B2B2B"

# Create video clip (showing first 15 seconds)
video_clip = Clip(
    asset=VideoAsset(
        id=video.id,
        start=0
    ),
    duration=15
)


# Add clip to the same track
track = Track()
track.add_clip(0, video_clip)

timeline.add_track(track)

# Generate and play
stream_url = timeline.generate_stream()
print(f"✅ Stream generated: {stream_url}")
play_stream(stream_url)

✅ Stream generated: https://play.videodb.io/v1/881c6055-c8c0-4419-a1d1-c4dab0418e28.m3u8


### Lets index the spoken content of the video first, in order to use caption asset.

In [9]:
video.index_spoken_words()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:41<00:00,  2.40it/s]


---

## 📦 Step 5: Basic Auto-Generated Captions

Let's create our first captioned video using the simplest approach: auto-generated subtitles with default styling.

### How It Works

When you set `src="auto"`, VideoDB automatically:
1. Transcribes the speech in your video
2. Generates time-synced subtitle text
3. Renders the captions on your video

### The Workflow

1. Create a **VideoAsset** with your video
2. Create a **CaptionAsset** with `src="auto"`
3. Wrap both in **Clips** with matching durations
4. Add both clips to the **same Track** at the same start time (0)
5. Generate the stream

This "horizontal stacking" approach layers the captions on top of the video.

In [10]:
# Initialize timeline with neutral gray background
timeline = Timeline(conn)
timeline.background = "#2B2B2B"

# Create video clip (showing first 15 seconds)
video_clip = Clip(
    asset=VideoAsset(
        id=video.id,
        start=60
    ),
    duration=15
)

# Create caption clip with auto-generation
caption_clip = Clip(
    asset=CaptionAsset(
        src="auto"  # This triggers automatic subtitle generation
    ),
    duration=15  # Must match video duration
)

# Add both clips to the same track
track = Track()
track.add_clip(0, video_clip)
track.add_clip(0, caption_clip)  # Stacked on top of video

timeline.add_track(track)

# Generate and play
stream_url = timeline.generate_stream()
print(f"✅ Stream generated: {stream_url}")
play_stream(stream_url)

✅ Stream generated: https://play.videodb.io/v1/66eac9ff-7609-436b-b953-fd3565a7cae5.m3u8


### What You See

The video now has subtitles! By default, you get:
- White text with black outline
- Positioned at middle-center of the screen
- Clear Sans font
- No animation (static subtitles)

Next, let's customize the appearance.

---

## 📦 Step 6: Caption Positioning & Font Styling

Now let's customize where captions appear and how they look.

### Positioning

Use the `Positioning` class with `CaptionAlignment` to place captions in different zones:
- `top_center`, `top_left`, `top_right`
- `middle_center`, `middle_left`, `middle_right`
- `bottom_center`, `bottom_left`, `bottom_right`

You can also set margins (left, right, vertical) to fine-tune placement.

### Font Styling

Use the `FontStyling` class to control:
- `size`: Font size in points
- `bold`: Make text bold
- `italic`: Italicize text
- `name`: Choose a font family

Let's create captions at the bottom with larger, bold text:

In [12]:
timeline = Timeline(conn)
timeline.background = "#2B2B2B"

video_clip = Clip(
    asset=VideoAsset(id=video.id, start=60),
    duration=15
)

# Captions with custom positioning and font
caption_clip = Clip(
    asset=CaptionAsset(
        src="auto",
        position=Positioning(
            alignment=CaptionAlignment.bottom_center,
            margin_v=50  # 50px from bottom
        ),
        font=FontStyling(
            size=48,
            bold=True,
            name = "Clear Sans",
        )
    ),
    duration=15
)

track = Track()
track.add_clip(0, video_clip)
track.add_clip(0, caption_clip)

timeline.add_track(track)

stream_url = timeline.generate_stream()
print(f"✅ Styled captions at bottom: {stream_url}")
play_stream(stream_url)

✅ Styled captions at bottom: https://play.videodb.io/v1/6dbba9aa-fcc2-426d-8ae4-f53300d4a2ad.m3u8


### Colors in CaptionAsset: ASS Format

CaptionAsset uses **ASS (Advanced SubStation Alpha)** color format, which differs from standard HTML hex codes.

**ASS Format**: `&HAABBGGRR` (Alpha-Blue-Green-Red order)

> **Note on Alpha**: In ASS, `00` is fully opaque and `FF` is fully transparent.

**Corrected Examples (Fully Opaque):**
- **White**: `&H00FFFFFF`
- **Yellow**: `&H0000FFFF`
- **Red**: `&H000000FF`
- **Blue**: `&H00FF0000`
- **Black**: `&H00000000`

You'll use these colors in the next sections for animations.

---

## 📦 Step 7: Animation — Reveal

Let's add our first animation: **Reveal**.

### What is Reveal?

The `reveal` animation makes text appear **word-by-word** as it's spoken. Each word becomes visible in sync with the audio.

This creates a dynamic, engaging subtitle experience where viewers see words appear progressively.

### How to Use It

Set `animation=CaptionAnimation.reveal` in your CaptionAsset.

In [13]:
timeline = Timeline(conn)
timeline.background = "#2B2B2B"

video_clip = Clip(
    asset=VideoAsset(id=video.id, start=60),
    duration=15
)

# Caption with reveal animation
caption_clip = Clip(
    asset=CaptionAsset(
        src="auto",
        animation=CaptionAnimation.reveal,
        position=Positioning(
            alignment=CaptionAlignment.bottom_center
        ),
        font=FontStyling(size=44, bold=True)
    ),
    duration=15
)

track = Track()
track.add_clip(0, video_clip)
track.add_clip(0, caption_clip)

timeline.add_track(track)

stream_url = timeline.generate_stream()
print(f"✅ Reveal animation applied: {stream_url}")
play_stream(stream_url)

✅ Reveal animation applied: https://play.videodb.io/v1/59ebded1-e922-4f10-abfd-148f51e761f9.m3u8


**Tip**: Reveal works especially well for dramatic narration or storytelling videos where you want to build suspense word-by-word.

---

## 📦 Step 8: Animation — Karaoke

Next up: **Karaoke** animation.

### What is Karaoke?

The `karaoke` animation changes the color of each word as it's spoken, creating a karaoke-style highlighting effect.

- **`primary_color`**: The default text color (before the word is spoken)
- **`secondary_color`**: The highlight color (when the word is active)

As the video plays, words transition from primary to secondary color in sync with speech.

### Color Format Reminder

Use ASS format:
- White text: `&H00FFFFFF`
- Yellow highlight: `&H0000FFFF`

In [14]:
timeline = Timeline(conn)
timeline.background = "#2B2B2B"

video_clip = Clip(
    asset=VideoAsset(id=video.id, start=60),
    duration=15
)

# Karaoke animation with color transition
caption_clip = Clip(
    asset=CaptionAsset(
        src="auto",
        animation=CaptionAnimation.karaoke,
        primary_color="&H00FFFFFF",  # White (default)
        secondary_color="&H0000FFFF",  # Yellow (active word)
        position=Positioning(
            alignment=CaptionAlignment.bottom_center
        ),
        font=FontStyling(size=44, bold=True)
    ),
    duration=15
)

track = Track()
track.add_clip(0, video_clip)
track.add_clip(0, caption_clip)

timeline.add_track(track)

stream_url = timeline.generate_stream()
print(f"✅ Karaoke animation applied: {stream_url}")
play_stream(stream_url)

✅ Karaoke animation applied: https://play.videodb.io/v1/97dd667c-061b-4bd8-b8ac-d57e692ae95c.m3u8


**Tip**: Karaoke is perfect for music videos, language learning content, or any video where you want to emphasize the active word being spoken.

---

## 📦 Step 9: Animation — Supersize

Let's explore the **Supersize** animation.

### What is Supersize?

The `supersize` animation makes the **active word grow larger** as it's spoken, then returns to normal size.

This creates visual emphasis on the currently spoken word through size change rather than color.

### When to Use It

Supersize works great for:
- Educational content where you want to draw attention to key terms
- Energetic, dynamic videos
- Content where color-based highlighting might clash with your video's color scheme

In [15]:
timeline = Timeline(conn)
timeline.background = "#2B2B2B"

video_clip = Clip(
    asset=VideoAsset(id=video.id, start=60),
    duration=15
)

# Supersize animation
caption_clip = Clip(
    asset=CaptionAsset(
        src="auto",
        animation=CaptionAnimation.supersize,
        primary_color="&H00FFFFFF",  # White text
        secondary_color="&H00356BFF",  # Orange for emphasis
        position=Positioning(
            alignment=CaptionAlignment.bottom_center
        ),
        font=FontStyling(size=44, bold=True)
    ),
    duration=15
)

track = Track()
track.add_clip(0, video_clip)
track.add_clip(0, caption_clip)

timeline.add_track(track)

stream_url = timeline.generate_stream()
print(f"✅ Supersize animation applied: {stream_url}")
play_stream(stream_url)

✅ Supersize animation applied: https://play.videodb.io/v1/bfd4c2cf-c9c3-4fb6-a23a-b7af59da67a0.m3u8


**Note**: The `secondary_color` in supersize controls the color of the enlarged word, creating both size and color emphasis.

---

## 📦 Step 10: Animation — Box Highlight

Finally, let's look at **Box Highlight**.

### What is Box Highlight?

The `box_highlight` animation draws a **box around the active word** as it's spoken. The box moves along with the speech, highlighting each word in sequence.

This creates a clear visual indicator that follows the speech, making it very easy for viewers to track which word is currently being spoken.

### Use Cases

Box highlight is excellent for:
- Accessibility (helping viewers follow along)
- Language learning (clear word-by-word tracking)
- Fast-paced content where viewers need help keeping up

In [16]:
timeline = Timeline(conn)
timeline.background = "#2B2B2B"

video_clip = Clip(
    asset=VideoAsset(id=video.id, start=60),
    duration=15
)

# Box highlight animation
caption_clip = Clip(
    asset=CaptionAsset(
        src="auto",
        animation=CaptionAnimation.box_highlight,
        primary_color="&H00FFFFFF",  # White text
        secondary_color="&H0000FF00",  # Green for box
        position=Positioning(
            alignment=CaptionAlignment.bottom_center
        ),
        font=FontStyling(size=44, bold=True)
    ),
    duration=15
)

track = Track()
track.add_clip(0, video_clip)
track.add_clip(0, caption_clip)

timeline.add_track(track)

stream_url = timeline.generate_stream()
print(f"✅ Box highlight animation applied: {stream_url}")
play_stream(stream_url)

✅ Box highlight animation applied: https://play.videodb.io/v1/69f091fe-1295-40de-8373-704df676e30e.m3u8


**Tip**: The `secondary_color` determines the color of the highlight box that follows the active word.

---

## 📦 Step 11: Advanced Styling with Borders and Shadows

Let's take caption styling to the next level by adding borders and backgrounds.

### BorderAndShadow Class

The `BorderAndShadow` class controls:
- **outline**: Width of the text outline
- **outline_color**: Color of the outline (ASS format)
- **shadow**: Drop shadow depth
- **style**: Border style (use `CaptionBorderStyle.outline_box` for a solid background box)

### Creating Professional Captions

Here's a complete example combining everything we've learned: positioning, font styling, colors, animation, and borders.

In [17]:
timeline = Timeline(conn)
timeline.background = "#2B2B2B"

video_clip = Clip(
    asset=VideoAsset(id=video.id, start=60),
    duration=15
)

# Professional captions with all styling options
caption_clip = Clip(
    asset=CaptionAsset(
        src="auto",
        animation=CaptionAnimation.karaoke,
        primary_color="&H00FFFFFF",  # White
        secondary_color="&H0000FFFF",  # Yellow
        back_color="&H00000000",  # Transparent background
        position=Positioning(
            alignment=CaptionAlignment.bottom_center,
            margin_v=60
        ),
        font=FontStyling(
            size=48,
            bold=True,
            name="Clear Sans"
        ),
        border=BorderAndShadow(
            style=CaptionBorderStyle.outline_and_shadow,
            outline=5,
            outline_color="&H00000000",  # Black outline
            shadow=3
        )
    ),
    duration=15
)

track = Track()
track.add_clip(0, video_clip)
track.add_clip(0, caption_clip)

timeline.add_track(track)

stream_url = timeline.generate_stream()
print(f"✅ Professional styled captions: {stream_url}")
play_stream(stream_url)

✅ Professional styled captions: https://play.videodb.io/v1/ca800a01-8f0a-42da-819f-cdc8688fe16d.m3u8


### Boxed Captions

For captions with a solid background box (like you see on TV broadcasts), use `CaptionBorderStyle.opaque_box`:

In [34]:
timeline = Timeline(conn)
timeline.background = "#2B2B2B"

video_clip = Clip(
    asset=VideoAsset(id=video.id, start=60),
    duration=15
)

# Captions with solid background box
caption_clip = Clip(
    asset=CaptionAsset(
        src="auto",
        animation=CaptionAnimation.reveal,
        primary_color="&H00FFFFFF",
        secondary_color="&H00FFFFFF",
        back_color="&H80000000",  # Semi-transparent black box
        position=Positioning(
            alignment=CaptionAlignment.bottom_center
        ),
        font=FontStyling(size=44, bold=True),
        border=BorderAndShadow(
            style=CaptionBorderStyle.opaque_box,
            outline=3
        )
    ),
    duration=15
)

track = Track()
track.add_clip(0, video_clip)
track.add_clip(0, caption_clip)

timeline.add_track(track)

stream_url = timeline.generate_stream()
print(f"✅ Boxed captions: {stream_url}")
play_stream(stream_url)

✅ Boxed captions: https://play.videodb.io/v1/e84579bb-46ed-4277-be6d-9d2789940137.m3u8


## 🎨 Caption Style Presets

Ready-to-use professional caption styles for different content types and platforms. Each preset combines specific fonts, colors, animations, and positioning to create distinctive looks.

---

### 1️⃣ TikTok Classic

The most recognizable caption style on TikTok - white bold text with thick black outline. Perfect for viral short-form videos where readability is crucial.

**Preset Parameters:**
- **Font:** Arial, 58pt, Bold
- **Position:** Bottom Center
- **Animation:** Impact
- **Primary Color:** `&H00FFFFFF` (White)
- **Secondary Color:** `&H0000FFFF` (Yellow highlight)
- **Back Color:** `&H80000000` (Semi-transparent black shadow)
- **Outline:** Black, 3pt width

In [23]:
# TikTok Classic Preset
timeline = Timeline(conn)

video_clip = Clip(
    asset=VideoAsset(id=video.id, start=60),
    duration=15
)

# Caption with TikTok Classic styling
caption_clip = Clip(
    asset=CaptionAsset(
        src="auto",
        font=FontStyling(
            name="Arial",
            size=58,
            bold=True
        ),
        position=Positioning(alignment=CaptionAlignment.bottom_center),
        animation=CaptionAnimation.color_highlight,
        primary_color="&H00FFFFFF",      # White text
        secondary_color="&H0000FFFF",    # Yellow highlight
        back_color="&H80000000",         # Semi-transparent black shadow
        border=BorderAndShadow(
            outline=3,
            outline_color="&H00000000"   # Black outline
        )
    ),
    duration=15
)

track = Track()
track.add_clip(0, video_clip)
track.add_clip(0, caption_clip)
timeline.add_track(track)

stream_url = timeline.generate_stream()
print("✅ TikTok Classic style applied!")
play_stream(stream_url)

✅ TikTok Classic style applied!


---

### 2️⃣ Cinematic Gold

Elegant golden italic text with smooth reveal animation - gives your content a premium movie subtitle feel. Ideal for storytelling, documentaries, or luxury brand content.

**Preset Parameters:**
- **Font:** Georgia, 52pt, Italic
- **Position:** Bottom Center
- **Animation:** Reveal
- **Primary Color:** `&H0000D7FF` (Golden yellow)
- **Secondary Color:** `&H0000FFFF` (Bright yellow)
- **Back Color:** `&H00000000` (Transparent)
- **Outline:** Black, 2pt width

In [24]:
# Cinematic Gold Preset
timeline = Timeline(conn)

video_clip = Clip(
    asset=VideoAsset(id=video.id, start=60),
    duration=15
)

# Caption with Cinematic Gold styling
caption_clip = Clip(
    asset=CaptionAsset(
        src="auto",
        font=FontStyling(
            name="Georgia",
            size=52,
            italic=True
        ),
        position=Positioning(alignment=CaptionAlignment.bottom_center),
        animation=CaptionAnimation.reveal,
        primary_color="&H0000D7FF",      # Golden yellow
        secondary_color="&H0000FFFF",    # Bright yellow
        back_color="&H00000000",         # Transparent
        border=BorderAndShadow(
            outline=2,
            outline_color="&H00000000"   # Black outline
        )
    ),
    duration=15
)

track = Track()
track.add_clip(0, video_clip)
track.add_clip(0, caption_clip)
timeline.add_track(track)

stream_url = timeline.generate_stream()
print("✅ Cinematic Gold style applied!")
play_stream(stream_url)

✅ Cinematic Gold style applied!


---

### 3️⃣ Bold Impact

Large white text that punches in with impact animation - maximum attention-grabbing power. Perfect for motivational content, key statements, or dramatic reveals centered on screen.

**Preset Parameters:**
- **Font:** Arial, 62pt, Bold
- **Position:** Middle Center
- **Animation:** Impact
- **Primary Color:** `&H00FFFFFF` (White)
- **Secondary Color:** `&H00FFFFFF` (White)
- **Back Color:** `&H00000000` (Transparent)
- **Outline:** Black, 4pt width

In [29]:
# Bold Impact Preset
timeline = Timeline(conn)
timeline.resolution=("608x1080")

video_clip = Clip(
    asset=VideoAsset(id=video.id, start=60),
    duration=15
)

# Caption with Bold Impact styling
caption_clip = Clip(
    asset=CaptionAsset(
        src="auto",
        font=FontStyling(
            name="Arial",
            size=62,
            bold=True
        ),
        position=Positioning(alignment=CaptionAlignment.middle_center),
        animation=CaptionAnimation.impact,
        primary_color="&H00FFFFFF",      # White
        secondary_color="&H00FFFFFF",    # White
        back_color="&H00000000",         # Transparent
        border=BorderAndShadow(
            outline=4,
            outline_color="&H00000000"   # Black outline
        )
    ),
    duration=15
)

track = Track()
track.add_clip(0, video_clip)
track.add_clip(0, caption_clip)
timeline.add_track(track)

stream_url = timeline.generate_stream()
print("✅ Bold Impact style applied!")
play_stream(stream_url)

✅ Bold Impact style applied!


---

### 4️⃣ Box Highlight

Words highlighted with animated purple boxes as they're spoken - the trending Instagram Reels and YouTube Shorts style. Creates dynamic emphasis that keeps viewers engaged.

**Preset Parameters:**
- **Font:** Verdana, 50pt, Bold
- **Position:** Bottom Center
- **Animation:** Box Highlight
- **Primary Color:** `&H00FFFFFF` (White text)
- **Secondary Color:** `&H00FF00FF` (Purple/Magenta box)
- **Back Color:** `&H00000000` (Transparent)
- **Outline:** Black, 2pt width

In [30]:
# Box Highlight Preset
timeline = Timeline(conn)

video_clip = Clip(
    asset=VideoAsset(id=video.id, start=60),
    duration=15
)

# Caption with Box Highlight styling
caption_clip = Clip(
    asset=CaptionAsset(
        src="auto",
        font=FontStyling(
            name="Verdana",
            size=50,
            bold=True
        ),
        position=Positioning(alignment=CaptionAlignment.bottom_center),
        animation=CaptionAnimation.box_highlight,
        primary_color="&H00FFFFFF",      # White text
        secondary_color="&H00FF00FF",    # Purple/Magenta box
        back_color="&H00000000",         # Transparent
        border=BorderAndShadow(
            outline=2,
            outline_color="&H00000000"   # Black outline
        )
    ),
    duration=15
)

track = Track()
track.add_clip(0, video_clip)
track.add_clip(0, caption_clip)
timeline.add_track(track)

stream_url = timeline.generate_stream()
print("✅ Box Highlight style applied!")
play_stream(stream_url)

✅ Box Highlight style applied!


---

### 5️⃣ Modern Boxed

White text in solid black boxes with reveal animation - sleek, minimalist Instagram story style. Creates clean, professional look with high contrast for maximum legibility.

**Preset Parameters:**
- **Font:** Verdana, 44pt, Bold
- **Position:** Bottom Center
- **Animation:** Reveal
- **Primary Color:** `&H00FFFFFF` (White text)
- **Secondary Color:** `&H00FFFFFF` (White)
- **Back Color:** `&HE0000000` (Nearly opaque black background)
- **Outline:** Black, 0pt width (no outline)

In [31]:
# Modern Boxed Preset
timeline = Timeline(conn)

video_clip = Clip(
    asset=VideoAsset(id=video.id, start=60),
    duration=15
)

# Caption with Modern Boxed styling
caption_clip = Clip(
    asset=CaptionAsset(
        src="auto",
        font=FontStyling(
            name="Verdana",
            size=44,
            bold=True
        ),
        position=Positioning(alignment=CaptionAlignment.bottom_center),
        animation=CaptionAnimation.reveal,
        primary_color="&H00FFFFFF",      # White text
        secondary_color="&H00FFFFFF",    # White
        back_color="&HE0000000",         # Nearly opaque black background
        border=BorderAndShadow(
            style=CaptionBorderStyle.opaque_box,
            outline=0
        )
    ),
    duration=15
)

track = Track()
track.add_clip(0, video_clip)
track.add_clip(0, caption_clip)
timeline.add_track(track)

stream_url = timeline.generate_stream()
print("✅ Modern Boxed style applied!")
play_stream(stream_url)

✅ Modern Boxed style applied!


---

### 📚 Additional Presets to Explore

Here are three more caption styles you can experiment with by using the same pattern shown above:

**6️⃣ Color Wave** (Karaoke-style)
- **Font:** Tahoma, 54pt, Bold
- **Position:** Bottom Center
- **Animation:** Color Highlight
- **Primary Color:** `&H00FFFFFF` (White base)
- **Secondary Color:** `&H005DE2F7` (Coral/orange highlight)
- **Back Color:** `&H00000000` (Transparent)
- **Outline:** Black, 2pt width

**7️⃣ Supersize Drama**
- **Font:** Trebuchet MS, 56pt, Bold
- **Position:** Middle Center
- **Animation:** Supersize
- **Primary Color:** `&H00FFFFFF` (White)
- **Secondary Color:** `&H0000FF00` (Green accent)
- **Back Color:** `&H00000000` (Transparent)
- **Outline:** Black, 3pt width

**8️⃣ Clean Minimal** (Documentary style)
- **Font:** Georgia, 46pt, Regular
- **Position:** Bottom Center
- **Animation:** None
- **Primary Color:** `&H00FFFFFF` (White)
- **Secondary Color:** `&H00FFFFFF` (White)
- **Back Color:** `&HA0000000` (Semi-transparent dark background)
- **Outline:** Black, 1pt width

---

## 🎬 Wrap-Up

Congratulations! You now know how to create professional, animated subtitles using CaptionAsset.

---

### What You Learned

1. **Auto-Generation**: Set `src="auto"` to automatically transcribe speech and generate subtitles
2. **Four Animation Types**:
   - **Reveal**: Word-by-word appearance
   - **Karaoke**: Color-changing text highlighting
   - **Supersize**: Active word grows larger
   - **Box Highlight**: Box follows active word
3. **ASS Color Format**: Use `&HAABBGGRR` format for colors
4. **Positioning**: Place captions anywhere (top, middle, bottom) with margins
5. **Font Styling**: Customize size, weight, and font family
6. **Borders & Shadows**: Add outlines, drop shadows, and background boxes

---

### Experiment Further

Try experimenting with:
- Different color combinations for your brand
- Mixing animations across different sections of your video
- Custom font families that match your style
- Different positions (top vs bottom) for different video types

**Happy captioning!** 🎉